In [1]:
import csv as csv
import re as re

In [2]:
def process_review(review):
    review = review.lower()
    review = re.sub('<br \/>|\(|\)|\.|\?|\,|\'|[0-9]|\!|\\\\|\"|\:|\;|\¨',' ',review)#regex for \ ?
    review = re.sub('\s[a-z]\s',' ',review)
    review = review.split()
    with open('stop-word-list.csv') as stopwords:
        for word in stopwords:
            stopword = word.split(', ')
        review = [word for word in review if word not in stopword]
        stopwords.close()
        
    return review


In [3]:
def update_array(review,sentiment,array):
    if sentiment=='1':
        for word in review:
            flag=0
            for i in range(len(array)):
                if array[i][0]==word:
                    flag=1
                    array[i][1] = array[i][1] + 1
                    break
            if flag!=1:
                array.append([word,1,0])
    else:
        for word in review:
            flag=0
            for i in range(len(array)):
                if array[i][0]==word:
                    flag=1
                    array[i][2] = array[i][2] + 1
                    break
            if flag!=1:
                array.append([word,0,1])
                
    return array    

In [4]:
def probability_calculations(review,sentiment,bag_of_words_array):
    prob = 1.0
    for word in review:
        flag=0
        for i in range(len(bag_of_words_array)):
                if bag_of_words_array[i][0]==word:
                    prob = prob*((bag_of_words_array[i][1]+1) if (sentiment==1) else (bag_of_words_array[i][2]+1))
                    prob = prob/(float)(bag_of_words_array[i][1]+bag_of_words_array[i][2]+2)
                    break
    return prob        

In [5]:
with open('labeledTrainData.tsv') as training_data:
    reader = csv.DictReader(training_data,delimiter='\t') #did not understand this delimeter
    c = [[0 for j in range(3)] for i in range(0)]
    flag = 0
    positive = 0
    negative = 0
    for line in reader:
        flag = flag+1
        line['review'] = process_review(line['review'])
        c = update_array(line['review'],line['sentiment'],c)
        if line['sentiment']=='1':
            positive =positive + 1
        else:
            negative = negative + 1
        if flag==12500:
            break
    print positive
    print negative            
        

6312
6188


In [8]:
with open('test.tsv') as test_data:
    read_test = csv.DictReader(test_data,delimiter='\t')
    test_array = []
    for line in read_test:
        line['review'] = process_review(line['review'])
        positive_prob = probability_calculations(line['review'],1,c)*(positive/(positive+negative))
        negative_prob = probability_calculations(line['review'],0,c)*(negative/(negative+positive))
        sentiment = 1 if positive_prob>negative_prob else 0
        #print line['sentiment'],sentiment,probability_calculations(line['review'],1,c),probability_calculations(line['review'],0,c)
        if str(sentiment)==(line['sentiment']):
            test_array.append(1)
        else:
            test_array.append(0)
    print sum(test_array)
    print len(test_array)
    print (float(sum(test_array))*100)/len(test_array)
        

2007
4001
50.1624593852
